In [9]:
# 훈련, 검증, 테스트 폴더로 이미지 복사하기

import os, shutil

original_dataset_dir = '.\\catdog\\train'

base_dir = '.\\catdog\\cats_and_dogs_small'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [10]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)] # 처음 1000개의 고양이 이미지를 train_cats_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500개의 고양이 이미지를 validation_cats_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500개의 고양이 이미지를 test_cats_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)] # 처음 1000개의 강아지 이미지를 train_dogs_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)] # 다음 500개의 강아지 이미지를 validation_dogs_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)] # 다음 500개의 강아지 이미지를 test_dogs_dir에 복사
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src,dst)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [30]:
# 분류 위한 소규모 컨브넷 만들기

model = Sequential() # 이 괄호 안에 add를 사용하여 layer를 추가하고 그것들을 다 연결하는 것
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='softmax'))
          
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)      

In [31]:
# 모델의 훈련 설정하기

from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [35]:
# ImageDataGenerator를 사용하여 디렉토리에서 이미지 읽기 
# (JPEG 콘텐츠를 RGB 픽셀 값으로 디코딩 후 부동 소수 타입의 텐서(넘파이 배열)로 변환)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# flow_from_directory() 메서드는 서브 디렉토리의 순서대로 레이블을 할당

# loss값을 binary_crossentropy를 쓰기 때문에 class_mode는 binary (이진 레이블 (개와 고양이만 구별))
# target_size=(150,150)는 그 크기로 변하게 해주는 것

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=20, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(150,150), batch_size=20, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [33]:
# 배치 제너레이터를 사용하여 모델 훈련하기

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, 
                              validation_data=validation_generator, validation_steps=50)

Epoch 1/30
100/100 [==============================] - 122s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 8.4333 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 118s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 9.1999 - val_acc: 0.5000
Epoch 3/30
100/100 [==============================] - 121s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 10.7333 - val_acc: 0.5000
Epoch 4/30
100/100 [==============================] - 119s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 10.7333 - val_acc: 0.5000
Epoch 5/30
100/100 [==============================] - 117s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 6.9000 - val_acc: 0.5000
Epoch 6/30
100/100 [==============================] - 117s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 9.1999 - val_acc: 0.5000
Epoch 7/30
100/100 [==============================] - 121s 1s/step - loss: 7.6666 - acc: 0.5000 - val_loss: 6.9000 - val_acc: 0.5000
Epoch 8/30
100/100 [==============================] - 126s 1s/step 

KeyboardInterrupt: 

In [ ]:
# 모델 평가하기 (훈련 데이터와 검증 데이터로 학습 후 맘에 들때까지 하이퍼 파리미터 (가중치 등) 조정하여 테스트 데이터로 평가)

print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=180)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# 모델 저장하기 

model.save('cats_and_dogs_small_1.h5')

In [ ]:
# 훈련의 정확도와 손실 그래프

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs,acc,'bo',label='Trainig acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# ImageDataGenerator를 사용하여 데이터 증식 설정하기

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, 
                             shear_range=0.1, zoom_range=0.1,horizontal_flip=True, fill_mode='nearest')

In [ ]:
# 랜덤하게 증식된 훈련 이미지 그리기

from keras.preprocessing import image 

fnames = sorted([os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)])

img_path = fnames[3] # 증식할 이미지 선택

img = image.load_img(img_path, target_size=(150,150)) # 이미지를 읽고 크기를 변환

x = image.img_to_array(img) # (150,150,3) 크기의 넘파이 배열로 변환
x = x.reshape((1,) + x.shape) # (1,150,150,3) 크기로 변환 (flow() 메서드는 배치 데이터를 기대하기 때문에 4D 텐서로 변환)

# 랜덤하게 변환된 이미지 배치 생성, 무한 반복 되기 때문에 여기선 4장만 만들고 중단
# 디렉토리에 클래스별로 예쁘게 구분을 잘 지어놨다면 flow_from_directory 를 사용하면 되겠지만, 그렇지 않을 땐 그냥 flow 쓴다.

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break
        
plt.show()

In [21]:
# 드롭아웃 포함한 새로운 컨브넷 정의하기

model = Sequential() # 이 괄호 안에 add를 사용하여 layer를 추가하고 그것들을 다 연결하는 것
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='softmax'))
          
model.summary()
          
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

SyntaxError: invalid syntax (<ipython-input-21-5fc2474f0fcb>, line 7)

In [ ]:
# 데이터 증식 제너레이터를 사용하여 데이터 준비

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255) # 검증 데이터는 증식 X

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=32,
                                                   class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary')

In [ ]:
# 모델 훈련하기

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=100, 
                              validation_data=validation_generator, validation_steps=50)

In [ ]:
# 모델 저장하기 

model.save('cats_and_dogs_small_2.h5')

In [ ]:
# 훈련의 정확도와 손실 그래프

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs,acc,'bo',label='Trainig acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()